# What proportion of Trump fundraising goes to legal fees?
## How else is donor money being spent, and what organizations does it flow through?

Inspiration:
* Economist: [Trump's legal fees are draining campaign funds](https://www.economist.com/graphic-detail/2024/02/01/donald-trumps-legal-fees-are-draining-his-campaign-funds)
* NYT: [Trump's PACs spent $50 million on legal expenses](https://www.nytimes.com/2024/01/30/us/politics/trump-legal-fees.html)

Goal:
* To produce an [alluvial diagram](https://flourish.studio/blog/animating-sankey-visualisations/) showing inputs from donors, pass-through organizatinos in the middle, and spending items at the end.

Data structure:
* Data should always be in *long format*. Columns labelled `source` and `target` indicate flow; for a multi-stage chart, columns labelled `Step from` and `Step to` indicate the stage.

So our data should roughly look like:

| Source    | Target | Amount | Step from | Step to |
| -------- | ------- | -------- | -------| -------- |
| Donors | MAGA Inc.   | $100   | 0     | 1     
| Maga Inc. | Save America     | $50 | 1 | 2
| Save America  | Legal fees  | $45 | 2 | 3

Another option is to use [Mike Bostock's Sankey chart component](https://observablehq.com/d/56c49c88e84b5de5) on Observable. This might be better for getting a clean and customizable SVG.

### Questions:
* How much money have Trump-affiliated entities (not including super PAC) raised this cycle?
* How is this money transferred between entities?
* How much of the money is spent on legal fees?
* What are the other main spending categories?

-> Note: when tallying fundraising, we must avoid double-counting money raised by one entity then transferred to another, which then also registers this as money raised.

## Step 1: make a list of entities

| Entity    | ID |
| -------- | ------- |
| Trump Save America JFC | C00770941
| Save America PAC | C00762591
| MAGA PAC | C00580100
| Donald J. Trump for President 2024 | C00828541
| MAGA Inc. | C00825851

Question: should we include MAGA Inc. in the calculation? Originally, Save America PAC sent it $60 million, but then [asked for a refund](https://www.nytimes.com/2023/07/31/us/politics/trump-pac-filing.html).

Initial assumption: we don't need to calculate receipts and disbursements for MAGA Inc. as an entity, but we should look for it in the list of receipts and disbursements for Save America.

## 2: Identify main disbursements and receipts for all entities

For each entity the download URL looks like:

https://www.fec.gov/data/receipts/?two_year_transaction_period=2024&data_type=processed&committee_id={ID}

https://www.fec.gov/data/disbursements/?two_year_transaction_period=2024&data_type=processed&committee_id={ID}

For some of the larger entities -- Trump Save America JFC and Donald J. Trump for President -- we cannot easily export the full list of receipts (both are ~1.5 million rows). But we don't actually need to: we can take the FEC's summary figures from the committee page, and also find the highest value transactions by sorting the data by the amount column, within the web view.

### Summary data:

[TRUMP SAVE AMERICA JOINT FUNDRAISING COMMITTEE](https://www.fec.gov/data/committee/C00770941/)
```
TOTAL RECEIPTS	$129,104,100.34
TOTAL CONTRIBUTIONS	$129,020,950.34
Total individual contributions	$128,925,548.34
Itemized individual contributions	$78,447,317.46
Unitemized individual contributions	$50,478,230.88

TOTAL DISBURSEMENTS	$129,329,597.07
OPERATING EXPENDITURES	$48,306,897.39
TRANSFERS TO AFFILIATED COMMITTEES	$80,872,394.99
```

[DONALD J. TRUMP FOR PRESIDENT 2024, INC.](https://www.fec.gov/data/committee/C00828541/)
```
TOTAL RECEIPTS	$75,811,055.89
TOTAL CONTRIBUTIONS	$503,378.32
TRANSFERS FROM OTHER AUTHORIZED COMMITTEES	$72,008,828.44

TOTAL DISBURSEMENTS	$45,744,016.42
OPERATING EXPENDITURES	$45,742,616.42
```
[SAVE AMERICA](https://www.fec.gov/data/committee/C00762591/)
```
TOTAL RECEIPTS	$52,285,425.36
TOTAL CONTRIBUTIONS	$84,794.36
TRANSFERS FROM AFFILIATED COMMITTEES	$9,889,462.00
CANDIDATE REFUNDS	$42,250,000.00


TOTAL DISBURSEMENTS	$65,409,102.32
OPERATING EXPENDITURES	$53,081,850.72
TRANSFERS TO AFFILIATED COMMITTEES	$11,750,000.00
```
[MAGA PAC](https://www.fec.gov/data/committee/C00580100/)
```
TOTAL RECEIPTS	$12,280,389.65
TRANSFERS FROM AFFILIATED COMMITTEES	$11,750,000.00

TOTAL DISBURSEMENTS	$11,919,311.86
OPERATING EXPENDITURES	$11,908,945.36
```
---

From the above, we can see that the Trump Save America JFC raises money almost entirely from donors. It transferred about two thirds of this money to other committees ($80.9 million), and spent the remaining third on operating expenses.

The official presidential campaign, Donald J. Trump for President, receives money almost entirely from committees ($72 million)

MAGA Pac has received almost all money from affiliated committees ($11.75 million)

Save America has received a mixture of direct transfers ($9.9 million) and "refunds" from MAGA Inc. super PAC ($42.25 million).


### Tracking the refunds

We ultimately want to understand where the money came from that Save America is using to pay legal fees. Since this money comes from "refunds" from MAGA Inc., we need to go back to the 2021-22 cycle to see what payments were made and where this money came from.

In 2021-22, the MAGA Inc. super PAC [received $60 million in payments from Save America](https://www.fec.gov/data/receipts/?committee_id=C00825851&two_year_transaction_period=2022&cycle=2022&line_number=F3X-11C&data_type=processed) according to FEC data. So where did Save America get this money in the first place?

[Save America 2021-22](https://www.fec.gov/data/committee/C00762591/?cycle=2022)
```
TOTAL RECEIPTS	$109,290,510.18
TOTAL CONTRIBUTIONS	$1,176,252.90
TRANSFERS FROM AFFILIATED COMMITTEES	$108,035,602.57

TOTAL DISBURSEMENTS	$122,196,336.99
OPERATING EXPENDITURES	$50,334,016.22
CONTRIBUTIONS TO OTHER COMMITTEES	$65,255,000.00
OTHER DISBURSEMENTS	$6,606,500.00

BEGINNING CASH ON HAND	$31,169,635.10
ENDING CASH ON HAND	$18,263,773.75
```

If we look at Save America's itemized [receipts for 2021-22](https://www.fec.gov/data/receipts/?cycle=2022&data_type=processed&committee_id=C00762591&two_year_transaction_period=2022) and sort by highest amount, we see that all top donations came from fundraising committees. 

List of top donors: `data/Save_America_PAC_top_donors-2021-2.csv`

Some donations were from the Trump Save America JFC, which is still Trump's main JFC. Other large donations came from Trump Make America Great Again Committee, which was Trump's JFC for the 2020 election.

In the 2020 election cycle the Trump Make America Great Again Committee [raised $883.7 million in individual contributions](https://www.fec.gov/data/committee/C00618371/?cycle=2020), and ended 2020 with $59.9 million cash on hand. In 2021-22, the Trump Make America Great Again Committee [received $29.3 million in donor payments](https://www.fec.gov/data/committee/C00618371/?cycle=2022), and spent $85 million in total, ending with $4.5 million cash on hand. The lion's share of the $72.2 million transferred to other committees went to Save America.

**Conclusion:**

We're interested in the 2023-24 cycle. In this cycle, we can list the value of the payment from MAGA Inc. to Save America, and explain that this came from donor money.

## 3: Calculate inputs and outputs for each entity

#### Trump Save America JFC | C00770941
```
INPUTS:
Total individual contributions: $128,925,548.34
```
```
OUTPUTS:
```

In [13]:
import pandas as pd
# Show all columns
pd.options.display.max_columns = None

In [14]:
tsa = pd.read_csv('data/TRUMP_SAVE_AMERICA_JFC-disbursements.csv')

In [36]:
# Group spending into categories and sum total amount
tsa.groupby(['disbursement_description'])['disbursement_amount'].sum()\
    .sort_values(ascending=False).head(25)

disbursement_description
TRANSFER OF NET PROCEEDS                                  80872394.99
DIRECT MAIL PRINTING & POSTAGE                            10490065.50
DIGITAL CONSULTING, ONLINE ADVERTISING & LIST RENTAL      10476686.76
MERCHANT MERCHANDISE FEES                                  4944661.19
MERCHANT FEES                                              4239584.82
DIGITAL CONSULTING, ONLINE ADVERTISING, & LIST RENTAL      3432984.08
SMS ADVERTISING                                            3406821.98
REIMBURSEMENT: DIGITAL CONSULTING & ONLINE ADVERTISING     2069945.56
DATA PROCESSING SERVICES                                   1862975.49
COMPLIANCE CONSULTING                                      1120395.75
EMAIL SERVICES                                              934223.70
LIST RENTAL                                                 694159.05
REIMBURSEMENT: EMAIL SERVICES                               500000.00
DIRECT MAIL PRINTING, POSTAGE, & LIST RENTAL                43630

In [38]:
# Which commmittees received transfers from TSA-JFC and how much?
tsa.query('disbursement_description == "TRANSFER OF NET PROCEEDS"').groupby('recipient_name')['disbursement_amount'].sum()

recipient_name
DONALD J. TRUMP FOR PRESIDENT 2024, INC.    72007996.40
SAVE AMERICA                                 8864398.59
Name: disbursement_amount, dtype: float64

#### DONALD J. TRUMP FOR PRESIDENT 2024, INC.
[C00828541](https://www.fec.gov/data/committee/C00828541/)

```
INPUTS:
TOTAL RECEIPTS	$75,811,055.89
TRANSFERS FROM OTHER AUTHORIZED COMMITTEES	$72,008,828.44
```
^ meaning almost all income is from TSA-JFC transfer

`OUTPUTS:`

In [39]:
djt = pd.read_csv('data/DJT_for_PRESIDENT-disbursements.csv')

/var/folders/wg/s1f24q713mb0h5lcj__zsrh1jz45r8/T/ipykernel_20882/4106574286.py:1: DtypeWarning: Columns (66,73) have mixed types. Specify dtype option on import or set low_memory=False.
  djt = pd.read_csv('data/DJT_for_PRESIDENT-disbursements.csv')


In [40]:
djt.groupby(['disbursement_description'])['disbursement_amount'].sum()\
    .sort_values(ascending=False).head(25)

disbursement_description
PLACED MEDIA                                                                         9373614.02
PAYROLL                                                                              4616184.07
TRAVEL: AIR                                                                          4550303.31
LEGAL CONSULTING                                                                     4044905.81
AUDIO VISUAL SERVICES, EVENT STAGING EXPENSE & TRAVEL EXPENSES                       2616272.66
CREDIT CARD PAYMENT: SEE ITEMIZATION(S) IF REQUIRED                                  2393348.40
POLITICAL STRATEGY CONSULTING                                                        1816503.23
DIGITAL CONSULTING, ONLINE ADVERTISING & LIST RENTAL                                 1783208.05
TRAVEL: LODGING                                                                      1236974.35
POLLING EXPENSE                                                                       978135.20
POLITICAL STRAT

#### SAVE AMERICA PAC
[C00762591](https://www.fec.gov/data/committee/C00762591/)

```
INPUTS:
CANDIDATE REFUNDS	$42,250,000.00 | MAGA INC.
TRANSFERS FROM AFFILIATED COMMITTEES	$9,889,462.00
```
[Detailed receipts downloaded here](https://www.fec.gov/data/receipts/?cycle=2024&data_type=processed&committee_id=C00762591&two_year_transaction_period=2024&min_amount=10000&line_number=F3X-12), filtered for transactions greater than $10k.

In [47]:
# Load receipts
sa_in = pd.read_csv('data/SAVE_AMERICA_PAC-receipts-10k.csv')

# What's the total?
sa_in['contribution_receipt_amount'].sum()

12987243.259999998

In [48]:
sa_in.groupby('contributor_name')['contribution_receipt_amount'].sum()4.1


contributor_name
TRUMP MAKE AMERICA GREAT AGAIN COMMITTEE                 1022982.15
TRUMP MAKE AMERICA GREAT AGAIN COMMITTEE - UNITEMIZED    3099862.52
TRUMP SAVE AMERICA JOINT FUNDRAISING COMMITTEE           8864398.59
Name: contribution_receipt_amount, dtype: float64

It seems like only the itemized figures are included in the transfer total, but why? 

Looking at the [receipt form](https://docquery.fec.gov/cgi-bin/fecimg/?202310199598633335), it seems like the donor name is listed as "TRUMP MAKE AMERICA GREAT AGAIN COMMITTEE - UNITEMIZED", which means the FEC web app is not linking the donation to ""TRUMP MAKE AMERICA GREAT AGAIN COMMITTEE" when calculating the total received from affiliated committees.

However it seems likely that this is, in fact, a donation that should be counted as coming from this committee. So we should add the two values together.

In [49]:
sa_in[sa_in['contributor_name'].str.contains("MAKE AMERICA GREAT AGAIN COMMITTEE")]\
    ['contribution_receipt_amount'].sum()

4122844.67

**Save America outputs**

In [52]:
sa_out = pd.read_csv('data/SAVE_AMERICA_PAC-disbursements.csv')

In [53]:
sa_out.groupby(['disbursement_description'])['disbursement_amount'].sum()\
    .sort_values(ascending=False).head(25)

disbursement_description
LEGAL CONSULTING                                                   44029620.11
TRANSFER TO AFFILIATED COMMITTEE                                   11750000.00
REIMBURSEMENT FOR LEGAL FEES                                        1696578.96
PAYROLL                                                             1180296.35
REIMBURSEMENT FOR LEGAL EXPENSES                                    1081768.52
DATA PROCESSING SERVICES                                             808073.47
RESEARCH CONSULTING                                                  696565.31
COMPLIANCE CONSULTING                                                670560.00
CONTRIBUTION REFUND                                                  565151.60
PAYROLL TAXES & WITHHOLDINGS                                         529173.92
LEGAL CONSULTING & TRAVEL EXPENSES                                   479265.69
TRAVEL: AIR                                                          454828.40
STRATEGY CONSULTING        

#### MAGA PAC
[C00580100](https://www.fec.gov/data/committee/C00580100/)

```
INPUTS:
TOTAL RECEIPTS	$12,280,389.65
TRANSFERS FROM AFFILIATED COMMITTEES	$11,750,000.00 (from Save America PAC)
```

```
OUTPUTS:
TOTAL DISBURSEMENTS	$11,919,311.86
OPERATING EXPENDITURES	$11,908,945.36
```

In [54]:
ma_out = pd.read_csv('data/MAGA_PAC-disbursements.csv')

In [55]:
ma_out.groupby(['disbursement_description'])['disbursement_amount'].sum()\
    .sort_values(ascending=False).head(25)

disbursement_description
LEGAL CONSULTING                                        1872159.00
DOCUMENT HOSTING AND CONSULTING                         1808991.54
REIMBURSEMENT FOR LEGAL FEES                            1409813.77
REIMBURSEMENT FOR LEGAL EXPENSES                         405481.20
COMPLIANCE CONSULTING                                    220000.00
REIMBURSEMENT FOR LEGAL FEES & COMPLIANCE CONSULTING     207364.85
TAXES                                                     16615.18
REIMBURSEMENT FOR LEGAL FEES & LEGAL EXPENSES             16418.40
CREDIT CARD PAYMENT: SEE ITEMIZATION(S) IF REQUIRED       13576.85
COMPLIANCE CONSULTING & DATA PROCESSING SERVICES          10192.00
ADMINISTRATIVE CONSULTING                                  9800.00
AMEX PMT: LEGAL CONSULTING                                 4437.00
AMEX PMT: WEB HOSTING SERVICES                             3400.39
DATA PROCESSING SERVICES                                   3280.20
AMEX PMT: SOFTWARE SERVICES          

Sidenote: the company that provides document hosting services, "2M DOCUMENT MANAGEMENT AND IMAGING, LLC", is [mentioned in this transcript](https://www.govinfo.gov/content/pkg/GPO-J6-TRANSCRIPT-CTRL0000916072/pdf/GPO-J6-TRANSCRIPT-CTRL0000916072.pdf) of the Jan 6th hearing.

#### MAGA INC. SUPER PAC
[C00825851](https://www.fec.gov/data/committee/C00825851/)

```
INPUTS:
TOTAL RECEIPTS	$62,448,648.35
TOTAL CONTRIBUTIONS	$59,295,656.71
    Total individual contributions	$58,044,123.65
    Other committee contributions	$1,251,533.06

OUTPUTS:
TOTAL DISBURSEMENTS	$93,327,186.42
OPERATING EXPENDITURES	$7,028,492.98
INDEPENDENT EXPENDITURES	$43,818,693.44
TOTAL CONTRIBUTION REFUNDS	$42,350,000.00
    Individual refunds	$100,000.00
    Other committee refunds	$42,250,000.00
```

So for MAGA Inc., the refund to Save America has been almost as much as it spent on campaign activities in 2023.

I'm not sure if we need to show what else MAGA Inc. spent money on, as it's independent expenditure.

## Test Observable chart
[Trump's spending](https://observablehq.com/d/56c49c88e84b5de5)